<a href="https://colab.research.google.com/github/thedarredondo/data-science-fundamentals/blob/main/Unit1/RosenbluthEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import pymc as pm
import xarray as xr
import arviz as az
from scipy import stats as stats
from matplotlib import pyplot as plt

The following implemantation of the Rosenbluth/metropolis algorithm was copied from [this repository](https://github.com/dustinstansbury/statistical-rethinking-2023/blob/b0d0f17f391c071ae9b49f5e9750de1f037d27b5/Lecture%2008%20-%20Markov%20Chain%20Monte%20Carlo.ipynb).

The oringinal creator and implementor of the metroplois algorithm was [Arianna Rosenbluth](https://imstat.org/2021/02/23/obituary-arianna-rosenbluth-1927-2020/)

In [3]:
def simulate_markov_visits(island_sizes, n_steps=100_000):
    """aka Metropolis algorithm"""

    # Metropolis algorithm
    island_idx = np.arange(len(island_sizes))
    visits = {ii: 0 for ii in island_idx}
    current_island_idx = np.random.choice(island_idx)
    markov_chain = []  # store history
    for _ in range(n_steps):

        # 1. Flip a coin to propose a direction, left or right
        coin_flip = np.random.rand() > 0.5
        direction = -1 if coin_flip else 1
        proposal_island_idx = np.roll(island_idx, direction)[current_island_idx]

        # 2. Proposal island size
        proposal_size = island_sizes[proposal_island_idx]

        # 3. Current island size
        current_size = island_sizes[current_island_idx]

        # 4. Go to proposal island if ratio of sizes is greater than another coin flip
        p_star = proposal_size / current_size
        move = np.random.rand() < p_star
        current_island_idx = proposal_island_idx if move else current_island_idx

        # 5. tally visits and repeat
        visits[current_island_idx] += 1
        markov_chain.append(current_island_idx)

    # Visualization
    island_idx = visits.keys()
    island_visit_density = [v / n_steps  for v in visits.values()]
    island_size_density = island_sizes / island_sizes.sum()

    _, axs = plt.subplots(1, 2, figsize=(12, 4))

    plt.sca(axs[0])
    plt.plot(island_sizes, lw=3, color='C0')
    plt.xlabel("Island Index")
    plt.ylabel("Island Population");
    plt.xticks(np.arange(len(island_sizes)));

    plt.sca(axs[1])
    plt.plot(island_idx, island_size_density, color='C0', lw=3, label='Population')
    plt.bar(island_idx, island_visit_density, color='k', width=.4, label='Visits')

    plt.xlabel("Island Index")
    plt.ylabel("Density");
    plt.xticks(np.arange(len(island_sizes)));

    plt.legend()
    return markov_chain

def plot_markov_chain(markov_chain, **hist_kwargs):
    _, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
    plt.sca(axs[0])
    plt.plot(markov_chain[:300])
    plt.xlabel("visit number")
    plt.ylabel("island index")
    plt.title("Metropolis Algorithm Markov Chain\nFirst 300 Steps");

    plt.sca(axs[1])
    plt.hist(markov_chain, orientation='horizontal', density=True, **hist_kwargs);
    plt.title("Resulting Posterior\n(horizontal)");

Let's run the algorithm for 60 "weeks" or iterations, like we did in class.

In [ ]:
n_steps = 60
island_sizes = np.array([1.,2.,3.,4.,5.,6.])
island_sizes /= island_sizes.min() / 6

gaussian_markov_chain = simulate_markov_visits(island_sizes, n_steps)

In [ ]:
n_steps = 1000
island_sizes = np.array([1.,2.,3.,4.,5.,6.])
island_sizes /= island_sizes.min() / 6

gaussian_markov_chain = simulate_markov_visits(island_sizes, n_steps)

In [ ]:
n_steps = 10000
island_sizes = np.array([1.,2.,3.,4.,5.,6.])
island_sizes /= island_sizes.min() / 6

gaussian_markov_chain = simulate_markov_visits(island_sizes, n_steps)